<a href="https://colab.research.google.com/github/TALeonard/19ma573thomasleonard/blob/master/src/hw8_2_heston_pricing_v00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this assignment, we are tasked with finding the price of a call given the following parameters:

Heston Parameters:

* init_state = [$S_{0} = 100., v_{0} = .04$], $r$ = 0.05, $\kappa$ = 1.2, $\theta$ = 0.04, $\xi$ = 0.3, $\rho$ = 0.5

Call information:

* otype = 1, K = 100., maturity = 1., market_price = 15

Given the Heston model, we know the relevant characteristic function. Heston model assumes the asset price $S_{t}$ follows a sotchastic process described by

$$dS_{t} = rS_{t}dt + \sqrt{v_{t}}S_{t}dW_{1,t}$$

$$dv_{t} = \kappa (\theta - v_{t})dt + \xi \sqrt{v_{t}} (\rho dW_{1,t} + \bar \rho dW_{2,t})$$

Given this process, if we know the characteristic function of $\ln(S_{t})$ then we can use that with the Fourier transform to find the solution, under which we know that

$$C = S_{0}I_{1} - Ke^{-rT}I_{2}$$

where

$$I_{1}(\phi,\ln(K)) = \frac{1}{2} + \frac{1}{\pi}\int_{0}^{\infty}Re(\frac{e^{-iu\ln(K)}\phi(u-i)}{iu\phi(-i)})du$$

and
$$I_{2}(\phi,\ln(K)) = \frac{1}{2} + \frac{1}{\pi}\int_{0}^{\infty}Re(\frac{e^{-iu\ln(K)}\phi(u)}{iu})du$$

with $\phi$ as the characteristic function of $\ln(S_{T})$, and $Re(\cdot)$ the real portion of whatever its argument is.

Based on this, we need to identify the characteristic function of $\ln(S_{T})$ under the Heston model. Ali Hirsa's book *Computational Methods in Finance* outlines this result for us. Specifically, it states that

$$\phi(u) = \frac{e^{iu\ln(S_{0}) + iurt + \frac{\kappa \theta t (\kappa - i\rho\xi u)}{\xi^{2}}}} {(\cosh(\frac{\gamma t}{2})+ \frac{\kappa - i\rho\xi u}{\gamma}\sinh(\frac{\gamma t}{2}))^{\frac{2\kappa\theta}{\xi^{2}}}} * e^{\frac{-(u^{2}+iu)v_{0}} {\gamma coth(\frac{\gamma t}{2})+\kappa - i\rho\xi u}}$$

Where $\gamma = \sqrt{\xi^{2}(u^{2}+iu) + (\kappa - i\rho\xi u)^{2}}$

While this may LOOK foreboding, a lot of it is actually known. Any instances of $S_{0}, v_{0}, \rho, \xi, \kappa, t, r$, and $\theta$ are all known values. Specifically, $t$ is maturity = 1 and the rest of these are as defined by the initial parameters. Thus, we can pare down the characteristic function in that sense, and it's just a lot of computation. Opt to supply parameters into the overall function; gamma will be discussed later as it is annoying and I'm not sure what to do with it.

$$\phi(u) = \frac{e^{iu\ln(100) + 0.05ui + \frac{1.2*0.04 (1.2 - 0.5*0.3ui)}{0.3^{2}}}} {(\cosh(\frac{\gamma}{2})+ \frac{1.2 - 0.5*0.3ui}{\gamma}\sinh(\frac{\gamma}{2}))^{\frac{2*1.2*0.04}{0.3^{2}}}} * e^{\frac{-(u^{2}+iu)0.04} {\gamma \coth(\frac{\gamma}{2})+1.2 - 0.5*0.3 ui}}$$


***

$$ = \frac{e^{iu\ln(100) + 0.05ui + \frac{0.048 (1.2 - 0.15ui)}{0.09}}} {(\cosh(\frac{\gamma}{2})+ \frac{1.2 - 0.15ui}{\gamma}\sinh(\frac{\gamma}{2}))^{\frac{16}{15}}} * e^{\frac{-(u^{2}+iu)0.04} {\gamma \coth(\frac{\gamma}{2})+1.2 - 0.15 ui}}$$

However, what we DO need to do is apply it into the argument of the $Re(\cdot)$ function. End me, please.

Consider the characteristic function evaluated at $u-i$ and at $-i$, since the above is the function evaluated at $u$. These will allow us to substitute them into the two Integrals.

$$\phi(u-i) = \frac{e^{i(u-i)\ln(100) + 0.05(u-i)i + \frac{0.048 (1.2 - 0.15(u-i)i)}{0.09}}} {(\cosh(\frac{\gamma}{2})+ \frac{1.2 - 0.15(u-i)i}{\gamma}\sinh(\frac{\gamma}{2}))^{\frac{16}{15}}} * e^{\frac{-((u-i)^{2}+i(u-i))0.04} {\gamma \coth(\frac{\gamma}{2})+1.2 - 0.15 (u-i)i}}$$

$$ = \frac{e^{ui\ln(100)+\ln(100) + 0.05ui + 0.05 + \frac{0.048 (1.2 - 0.15ui - 0.15)}{0.09}}} {(\cosh(\frac{\gamma}{2})+ \frac{1.2 - 0.15ui - 0.15}{\gamma}\sinh(\frac{\gamma}{2}))^{\frac{16}{15}}} * e^{\frac{-(u^{2} - ui)0.04} {\gamma \coth(\frac{\gamma}{2})+1.2 - 0.15 (u-i)i}}$$

Here, we can split the fraction in the exponent of the numerator into 2 terms: $\frac{0.048*(1.2-0.15)}{0.09} = 0.56$ and $-\frac{0.048*0.15ui}{0.09} = -0.08ui$.

$$ \rightarrow = \frac{e^{ui(\ln(100) - 0.03) + \ln(100) + 0.61}} {(\cosh(\frac{\gamma}{2})+ \frac{1.05 - 0.15ui}{\gamma}\sinh(\frac{\gamma}{2}))^{\frac{16}{15}}} * e^{\frac{-(u^{2} - ui)0.04} {\gamma \coth(\frac{\gamma}{2}) + 1.05 - 0.15ui}}$$

Recall the definitions of $\cosh(\cdot)$, $\sinh(\cdot)$, and $\coth(\cdot)$:

* $\cosh(x) = \frac{e^{x} + e^{-x}}{2}$

* $\sinh(x) = \frac{e^{x} - e^{-x}}{2}$

* $\coth(x) = \frac{\cosh(x)}{\sinh(x)} = \frac{e^{x} + e^{-x}}{e^{x} - e^{-x}}$

Before we continue, consider the evaluation of $\gamma$:

$$\gamma = \sqrt{\xi^{2}(u^{2}+iu) + (\kappa - i\rho\xi u)^{2}} =  \sqrt{0.3^{2}(u^{2}+iu) + (1.2 - 0.5*0.3 ui)^{2}}$$

$$ = \sqrt{0.09(u^{2}+iu) + (1.2 - 0.15 ui)^{2}}$$

$$ = \sqrt{0.09(u^{2}+iu) + 1.44 - 0.36ui + 0.0225u^{2}i^{2}}$$

$$ = \sqrt{0.09u^{2} - 0.0225u^{2} +0.09ui - 0.36ui + 1.44}$$

$$ = \sqrt{0.0675u^{2} - 0.27ui + 1.44}$$

$$ = \sqrt{0.0225(3u^{2} - 12ui + 64)} $$

$$ = 0.15 \sqrt{3u^{2} - 12ui + 64}$$

Now, evaluate at $u = u-i$:

$$\gamma|_{u-i} = 0.15 \sqrt{3(u-i)^{2} - 12(u-i)i + 64}$$

$$ = 0.15 \sqrt{3(u^{2} - 2ui -1) - 12(ui+1) + 64}$$

$$ = 0.15 \sqrt{3u^{2} - 6ui - 3 - 12ui - 12 + 64}$$

$$ = 0.15 \sqrt{3u^{2} - 6ui - 12ui - 3 - 12 + 64}$$

$$ = 0.15 \sqrt{3u^{2} - 18ui + 49}$$

How do I deal with this? Ask prof. when he next responds.


On Gamma:

$$\gamma = \sqrt{\xi^{2}(u^{2}+iu) + (\kappa - i\rho\xi u)^{2}} =  \sqrt{0.3^{2}(u^{2}+iu) + (1.2 - 0.5*0.3 ui)^{2}}$$

$$ = \sqrt{0.09(u^{2}+iu) + (1.2 - 0.15 ui)^{2}}$$

$$ = \sqrt{0.09(u^{2}+iu) + 1.44 - 0.36ui + 0.0225u^{2}i^{2}}$$

$$ = \sqrt{0.09u^{2} - 0.0225u^{2} +0.09ui - 0.36ui + 1.44}$$

$$ = \sqrt{0.0675u^{2} - 0.27ui + 1.44}$$

$$ = \sqrt{0.0225(3u^{2} - 12ui + 64)} $$

$$ = 0.15 \sqrt{3u^{2} - 12ui + 64}$$

this seems like a lot of work...did he mean I could just jump to the end? Doubtful.